<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_asr_pipeline_optimisations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A walkthrough of the pleathora of optimisations and nifty features offered via `pipeline` for Automatic Speech Recognition! 🤗

from yours truly - [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb) 🤙

In [ ]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum bitsandbytes
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
%load_ext autotime

time: 299 µs (started: 2023-10-14 16:00:04 +00:00)


In [ ]:
import torch
from transformers import pipeline

time: 17.7 s (started: 2023-10-13 17:02:56 +00:00)


## Download a test file!

This is a 2 hour long audio snippet from a revdotcoms Earnings22 dataset.

In [ ]:
!wget https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac

--2023-10-14 16:00:04--  https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.24, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/96/e4/96e4f69cd112b019dd764318570e47e5fe96de53d8c32a99d745e72d9086e355/b2fd593ce144a8d904cf49a4ed77ed06eb50644a053dddd280c81a3ef94fb60e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_altman_lex_podcast_367.flac%3B+filename%3D%22sam_altman_lex_podcast_367.flac%22%3B&response-content-type=audio%2Fx-flac&Expires=1697558404&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzU1ODQwNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9lNC85NmU0ZjY5Y2QxMTJiMDE5ZGQ3NjQzMTg1NzBlNDdlNWZlOTZkZTUzZDhjMzJhOTlkNzQ1ZTcyZDkwODZl

## Inference

# Base case

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                device="cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 1min 39s (started: 2023-10-13 14:26:56 +00:00)


In [ ]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30)

time: 31min 1s (started: 2023-10-13 14:28:56 +00:00)


In [ ]:
outputs["text"][:200]

' We have been a misunderstood and badly mocked org for a long time. When we started, we announced the org at the end of 2015 and said we were going to work on AGI, people thought we were batshit insan'

time: 6.63 ms (started: 2023-10-13 15:00:48 +00:00)


It took about 25 minutes and 2 seconds to run inference on a 2 hour long snippet.

Can we make this faster?

## Cue.. Batching

In [ ]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=8)

time: 13min 19s (started: 2023-10-13 15:00:58 +00:00)


In [ ]:
outputs["text"][:200]

' We have been a misunderstood and badly mocked org for a long time. When we started, we announced the org at the end of 2015 and said we were going to work on AGI, people thought we were batshit insan'

time: 3.02 ms (started: 2023-10-13 15:17:24 +00:00)


Voila!! It only took the `pipeline` 13 minutes and 36 seconds to run inference this time. We were able to reduce the inference time by 50%. 🚀

But, the real question is - can we make it even faster? 💨

## Half - Precision

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 2min 16s (started: 2023-10-13 16:47:41 +00:00)


In [ ]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=16,
               return_timestamps=True)

time: 6min 13s (started: 2023-10-13 16:51:08 +00:00)


In [ ]:
outputs["text"][:200]

' We have been a misunderstood and badly mocked org for a long time. Like when we started, we like announced the org at the end of 2015 and said we were going to work on AGI, like people thought we wer'

time: 3.41 ms (started: 2023-10-13 16:58:55 +00:00)


## BetterTransformer

In [ ]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

pipe.model = pipe.model.to_bettertransformer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


time: 2min 17s (started: 2023-10-13 17:03:16 +00:00)


In [ ]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=16,
               return_timestamps=True)

time: 5min 42s (started: 2023-10-13 17:05:33 +00:00)


In [ ]:
outputs["text"][:200]

' We have been a misunderstood and badly mocked org for a long time. Like when we started, we like announced the org at the end of 2015 and said we were going to work on AGI, like people thought we wer'

time: 2.31 ms (started: 2023-10-13 17:11:17 +00:00)


## BetterTransformer + Higher batch size

In [ ]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=24,
               return_timestamps=True)

time: 5min 2s (started: 2023-10-13 17:11:22 +00:00)


In [ ]:
outputs["text"][:200]

' We have been a misunderstood and badly mocked org for a long time. Like when we started, we like announced the org at the end of 2015 and said we were going to work on AGI, like people thought we wer'

time: 3.87 ms (started: 2023-10-13 17:17:24 +00:00)


In [ ]:
import torch
from transformers import pipeline, BitsAndBytesConfig, WhisperForConditionalGeneration, WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2", quantization_config=quantization_config)

pipe = pipeline("automatic-speech-recognition",
                model=model,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 1min 57s (started: 2023-10-14 16:00:06 +00:00)


In [5]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=24,
               return_timestamps=True)

RuntimeError: ignored

time: 36.9 s (started: 2023-10-14 16:24:41 +00:00)
